---

#Projeto de Disciplina –  IMDb Filmes/ Filmes de 1970 a 2025

---
Universidade Federal de São Carlos

Curso: Bacharelado em Ciência da Computação de Sorocaba

Disciplina: Novas Tecnologias de Bancos de Dados

Professora: Profa. Dra. Sahudy Montenegro González

---

**Grupo 12**

**Integrantes:**

- Gabriel Evangelista Gonçalves da Silva (802791)
- Natalia dos Santos Carvalho (773383)
- Thiago Proença Idro (771064)

---

**Resumo:**

<de 3 a 4 linhas explicando o que o projeto faz>

---

## 1. Introdução

A indústria cinematográfica é um mercado de alto investimento e risco, onde decisões de produção, marketing e distribuição são baseadas em um complexo conjunto de variáveis. Para um gestor de estúdio, produtor ou analista de plataforma de streaming, entender o que define o "sucesso" na indústria de filmes é fundamental. No entanto, o sucesso pode ser medido de várias formas: é o retorno financeiro, a aclamação da crítica especializada, a popularidade com o público ou o reconhecimento da indústria?

Atualmente, as informações necessárias para responder a essas perguntas de forma integrada encontram-se isoladas em diferentes fontes de dados. Por exemplo, dados financeiros detalhados como orçamento e receita podem estar em uma base, enquanto dados de recepção crítica e prêmios estão em outra. Um gestor não consegue, de forma simples, cruzar o impacto do orçamento na nota da crítica ou a relação entre prêmios e a popularidade. Além disso, podemos correlacionar essas informações com informações menos objetivas dos filmes; avaliar essas métricas a partir de atores conhecidos ou gêneros de filme podem trazer uma visão detalhada acima do tema, observando peculiaridades da indústria de filmes.

Este projeto visa prototipar um Data Warehouse (DW) que aborda esse problema de negócio. O objetivo é integrar e consolidar essas múltiplas fontes de dados em um modelo multidimensional coeso. Com o DW implementado, será possível executar análises complexas (OLAP) para identificar padrões e correlações, focando em três eixos principais de análise: a rentabilidade financeira (ROI), a recepção crítica e o desempenho de prêmios e popularidade.



---

## 2. Requisitos do negócio
Para entender o sucesso dos filmes, temos os seguintes requisitos de negócios: 

###Requisito de Negócio 1 (Análise de Rentabilidade): 
Analisar o Retorno sobre Investimento (ROI) e o lucro médio dos filmes, segmentado por gênero e por data de lançamento. O objetivo é identificar quais categorias de filme são historicamente mais rentáveis e quais apresentam maior risco (baixo ROI). **[Natália]**

**Consulta**: Qual o ROI e o lucro médio por gênero e pela data de lançamento?

###Requisito do Negócio 2 (Crítica): 
Identificar a discrepância entre a recepção do público (nota IMDb e rotten tomatoes) e da crítica especializada (nota Metascore). Como essa correlação se distribui por Classificação Indicativa e país de produção? O objetivo é encontrar filmes 'cult' (amados pelo público, odiados pela crítica) ou 'armadilhas de crítico' (o oposto). **[Gabriel]**

**Consulta**: Qual a nota média IMDb, rotten tomatoes e metacritic por classificação indicativa e país de produção?

###Requisito do Negócio 3 (Desempenho): 
Queremos entender o perfil de sucesso dos talentos da indústria. Existe uma correlação entre a popularidade e os prêmios associados a cada pessoa? Ou seja, indivíduos que participam de filmes com alto score de popularidade também tendem a participar de filmes que ganham muitos prêmios? Como essa correlação se apresenta quando filtramos pela função da pessoa? **[Thiago]**

**Consulta**: Qual o score de popularidade e prêmios ganhos e nomeados de pessoas por suas funções?

---

## 3. Fontes dos dados
Para responder aos requisitos de negócio definidos, o projeto utilizará duas APIs como fontes de dados principais. Nenhuma fonte, isoladamente, foi capaz de fornecer todos os atributos necessários para os três pilares de análise. As fontes selecionadas são:

###TMDb API (The Movie Database):

**Domínio Principal:** Dados Financeiros.

**Justificativa:** Esta API é a fonte principal para as métricas de budget (Orçamento) e revenue (Receita), que são a base do Requisito de Negócio 1 (Análise de Rentabilidade). Além disso, contém o campo e popularity com um score de popularidade dos filmes, importante para o Requisito de Negócio 3 (Desempenho).

###OMDb API (Open Movie Database):

**Domínio Principal:** Dados de Recepção Crítica e Reconhecimento.

**Justificativa:** Esta API é a fonte principal para as métricas de imdb e  Metacritic. Além disso, contém o campo textual Awards (Prêmios). Estes atributos são essenciais para o Requisito de Negócio 2 (Crítica) e Requisito de Negócio 3 (Desempenho).

O processo de ETL irá consultar ambas as APIs para cada filme e unificar os resultados.

O cruzamento das informações será feito utilizando o imdb_id como chave de integração, que está presente em ambas as fontes.

Isso permitirá a criação de um modelo multidimensional coeso, onde as métricas financeiras (do TMDB) poderão ser analisadas contra as métricas de crítica (do OMDb) usando as mesmas dimensões (como Gênero, Ano de Lançamento, País, etc.), respondendo assim às consultas complexas que motivaram o projeto.

Os códigos implementados para a extração dos dados está disponível no link: https://github.com/thiagoproenca/IMDB-Filmes 


---

## 4. Camada Bronze

De acordo com os requisitos de negócio projetados, o fato tratado no data warehouse teve como foco métricas de orçamento utilizado, receita adquirida e os diversos tipos de avaliação adquiridas nas fontes de dados; seus atributos coerentes com as métricas observadas nos requisitos de negócio.

O modelo apresenta 3 dimensões: Filme, Gênero e Pessoa. O recurso de tabelas pontes é utilizado para representar a natureza de relações N para N entre filmes e gêneros, filmes e elenco (para os diversos tipos de cargo do elenco). O modelo apresenta também hierarquia nos atributos relacionados a tempo: ano_lancamento, mes_lancamento e dia_lancamento.

### Modelagem

[IMAGEM]

In [0]:
//Trazendo o arquivo parquet com a extração

### Análise Exploratória

aqui dá pra fazer uns sqls super básicos pra analises bem si ples, tipo botar as awards numa tabela

In [0]:
from pyspark.sql.functions import col, sum, when, explode
from pyspark.sql import SparkSession

In [0]:
#Código análise exploratória

from pyspark.sql.functions import col, sum, when
from pyspark.sql import SparkSession

# Iniciar sessão Spark (se ainda não estiver ativa)
spark = SparkSession.builder.appName("ParquetToTable").getOrCreate()

df_parquet = spark.read.parquet("/Volumes/workspace/public/data/extraction_file.parquet")
df_parquet.createOrReplaceTempView("minha_tabela_temp")

resultado = spark.sql("SELECT id, title, release_date, runtime, budget, revenue, genres, credits, awards, ratings  FROM minha_tabela_temp")
resultado.show()

+-------+--------------------+------------+-------+---------+---------+--------------------+--------------------+--------------------+--------------------+
|     id|               title|release_date|runtime|   budget|  revenue|              genres|             credits|              awards|             ratings|
+-------+--------------------+------------+-------+---------+---------+--------------------+--------------------+--------------------+--------------------+
|1084242|          Zootopia 2|  2025-11-26|    107|180000000|233000000|[Animação, Famíli...|{[{Judy Hopps (vo...|1 win & 2 nominat...|{7.8/10, 73/100, ...|
|1419406|  A Sombra Do Perigo|  2025-08-16|    142|        0|174400000|[Ação, Crime, Thr...|{[{Huang Dezhong,...|7 wins & 1 nomina...| {7.5/10, NULL, 80%}|
|1309012|             Altered|  2025-09-18|     85| 15000000|        0|[Ficção científic...|{[{Leon, Tom Felt...|                 N/A|{3.6/10, NULL, NULL}|
|1363123|  Plano em Família 2|  2025-11-11|    106|        0|   

In [0]:
rel2 = spark.sql("SELECT credits FROM minha_tabela_temp")
rel2.show()

+--------------------+
|             credits|
+--------------------+
|{[{Judy Hopps (vo...|
|{[{Huang Dezhong,...|
|{[{Leon, Tom Felt...|
|{[{Dan Morgan, Ma...|
|{[{Glinda, Ariana...|
|{[{Victor Franken...|
|{[{Vlad / Dracula...|
|{[{Nathan Mcclusk...|
|{[{Thia / Tessa, ...|
|{[{William Radfor...|
|{[{Jake Wilcott, ...|
|{[{Yuji itadori (...|
|{[{Bob, Leonardo ...|
|{[{Michelle, Emma...|
|{[{Tanjiro Kamado...|
|{[{J. Daniel Atla...|
|{[{Brian, Kevin J...|
|{[{Clara Grant, M...|
|{[{Rumi (voice), ...|
|{[{Leo, Mark Cole...|
+--------------------+
only showing top 20 rows


In [0]:
df_generos_separados = df_parquet.withColumn(
    "genre",  # Nome da nova coluna que irá conter um único gênero
    explode(col("genres")) # Aplica o explode na coluna 'genres'
).select(
    "id",
    "title",
    "runtime",
    "genre" # Seleciona a nova coluna separada
)

df_generos_separados.show()

+-------+--------------------+-------+-----------------+
|     id|               title|runtime|            genre|
+-------+--------------------+-------+-----------------+
|1084242|          Zootopia 2|    107|         Animação|
|1084242|          Zootopia 2|    107|          Família|
|1084242|          Zootopia 2|    107|          Comédia|
|1084242|          Zootopia 2|    107|         Aventura|
|1084242|          Zootopia 2|    107|         Mistério|
|1419406|  A Sombra Do Perigo|    142|             Ação|
|1419406|  A Sombra Do Perigo|    142|            Crime|
|1419406|  A Sombra Do Perigo|    142|         Thriller|
|1309012|             Altered|     85|Ficção científica|
|1309012|             Altered|     85|             Ação|
|1363123|  Plano em Família 2|    106|             Ação|
|1363123|  Plano em Família 2|    106|          Comédia|
| 967941|    Wicked: Parte II|    138|         Fantasia|
| 967941|    Wicked: Parte II|    138|         Aventura|
| 967941|    Wicked: Parte II| 

In [0]:
# 1. Aplicar o explode no array 'credits.cast'
df_cast_desaninhado = df_parquet.withColumn(
    "actor_struct", # Nova coluna que contém o Struct do ator
    explode(col("credits.cast")) 
)

# 2. Acessar os campos 'character' e 'name' dentro do Struct explodido
df_atores_separados = df_cast_desaninhado.select(
    col("id"),
    col("title"),
    col("actor_struct.name").alias("actor_name"),       # Acessando campo 'name'
)

df_atores_separados.display()

id,title,actor_name
1084242,Zootopia 2,Ginnifer Goodwin
1084242,Zootopia 2,Jason Bateman
1084242,Zootopia 2,Ke Huy Quan
1084242,Zootopia 2,Fortune Feimster
1084242,Zootopia 2,Andy Samberg
1084242,Zootopia 2,David Strathairn
1084242,Zootopia 2,Idris Elba
1084242,Zootopia 2,Shakira
1084242,Zootopia 2,Patrick Warburton
1084242,Zootopia 2,Quinta Brunson


## 5. Camada Prata

###Modelagem ETL

Para a extração de dados, uma mistura entre os dois sites (TMDb e OMDb) pode ser utilizada para alimentar os dados do data warehouse. A partir das API fornecidas, temos a disposição de dados de filme abaixo:

[IMAGEM]

Esse formato ainda dispõe da diferenciação entre tipos de cargo dentro do elenco, diferenciando atores, produtores, escritores e diretores como disposto abaixo:

[IMAGEM]

A partir da extração realizada até o momento de entrega, foi possível definir para a etapa de transformação e limpeza as seguintes operações: 

- Padronização de nome de gênero pós extração para formato coerente com o data warehouse **[Natália]**;
- Transformação no formato das premiações e nomeações de uma única string para dois números independentes **[Gabriel]**;
- Transformação de data de lançamento de uma única string para três números independentes **[Gabriel]**;
- Remoção de filmes sem métrica de avaliação **[Gabriel]**;
- Remoção de filmes sem métrica de orçamento e/ou lucro **[Natália]**;
- Remover filmes com valores em alfabetos diferentes **[Thiago]**;
- Padronizar avaliações em um único formato, de escala 10 ou porcentagem apenas para escala 10 **[Thiago]**.

In [0]:
//Transformações e limpezas

## 6. Camada Ouro

## 7. Bibliografia
	
IMDb. IMDb: Ratings, Reviews, and Where to Watch the Best Movies and TV Shows. Disponível em: https://www.imdb.com/pt/ (acesso em 04/11/2025).

TMDB. The Movie Database (TMDB). Disponível em: https://www.themoviedb.org/ (acesso em 04/11/2025).

OMDb API. The Open Movie Database (OMDb). Disponível em: https://www.omdbapi.com/ (acesso em 04/11/2025).

Metacritic. Movie Reviews, TV Reviews, Game Reviews, and Music Reviews - Metacritic. Disponível em: https://www.metacritic.com/ (acesso em 04/11/2025).

Rotten Tomatoes. Rotten Tomatoes: Movies | TV Shows | Movie Trailers | Reviews. Disponível em: https://www.rottentomatoes.com/ (acesso em 04/11/2025).